In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import *
from sklearn.cluster import KMeans

In [9]:
outputbasefolder = '../../../Algorithms/Unsupervised_Segmentation/Approaches/With_Scribble/Outputs/Human_DLPFC/Report'

In [17]:
import os
pr_dir  = '../../../../Algorithms/Unsupervised_Segmentation/Approaches/With_Scribble/Outputs/Human_DLPFC/151673/'
for dir_name in os.listdir(pr_dir):
    print(dir_name)

desktop.ini
Hexa_Filter_10_20_15_seed_6
Manual_Scribble_Miu_1
Manual_Scribble_Miu_10_Niu_10
Manual_Scribble_Miu_10_Niu_3
Manual_Scribble_Miu_10_Niu_5
Manual_Scribble_Miu_1_Niu_10
Manual_Scribble_Miu_1_Niu_3
Manual_Scribble_Miu_1_Niu_5
Manual_Scribble_Miu_20
Manual_Scribble_Miu_40
Manual_Scribble_Miu_5_Niu_10
Manual_Scribble_Miu_5_Niu_3
Manual_Scribble_Miu_5_Niu_5
Manual_Scribble_Miu_60
many_models_10_15_20_itr_1
many_models_10_15_20_itr_2
many_models_10_15_20_itr_3
many_models_10_15_20_itr_4
many_models_10_15_20_itr_5
many_models_1_0_0_itr_1
many_models_1_0_0_itr_2
many_models_1_0_0_itr_3
many_models_1_0_0_itr_4
many_models_1_0_0_itr_5
many_models_1_1_2_itr_1
many_models_1_1_2_itr_2
many_models_1_1_2_itr_3
many_models_1_1_2_itr_4
many_models_1_1_2_itr_5
mclust_miu_1
mclust_miu_10
mclust_miu_13
mclust_miu_15
mclust_miu_17
mclust_miu_19
mclust_miu_2
mclust_miu_3
mclust_miu_4
mclust_miu_5
mclust_miu_6
minlabel_1
minlabel_10_5_10_all_gene
minlabel_10_5_15_all_gene
minlabel_10_5_20_all_gene

In [10]:
co_ordinates = pd.read_csv('../Coordinates/coordinates.csv',index_col=0,header=0)
manual_annotation = pd.read_csv('../manual_annotations.csv',index_col=0,header=0)
pc_s = pd.read_csv('../Principal_Components/CSV/15_pcs.csv',index_col=0,header=0)
mclust_init = pd.read_csv('../../../../Algorithms/Unsupervised_Segmentation/Approaches/With_Scribble/Outputs/Human_DLPFC/151673/test_1/Outputs/Cluster_Labels_15final.csv',index_col=0,header=0)

c2 = (co_ordinates - co_ordinates.mean())*10/ co_ordinates.std()
# c2 = co_ordinates

combined_df = pd.concat([c2,manual_annotation,mclust_init,pc_s],axis=1)
combined_df['label'].fillna(-1,inplace=True)
# combined_df = combined_df.dropna(subset=['label'])
# combined_df['label'] = combined_df['manual_annotation']
combined_df.dropna(inplace=True)
# combined_df['label'] = combined_df['label'].astype(int)
# combined_df.drop(combined_df[combined_df['label']==40].index,inplace=True)


In [6]:
label_map = {
    0: 'WM',
    6: 'L1',
    5: 'L2',
    4: 'L3',
    3: 'L4',
    2: 'L5',
    1: 'L6',
    7: 'L7',
    -1: 'undetected',
}
combined_df['label'] = combined_df['label'].map(label_map)

In [16]:
for i in combined_df['manual_annotation'].unique():
    filtered_df = combined_df[combined_df['manual_annotation']==i]
    if filtered_df.shape[0] < 2:
        continue
    max_score = -2
    max_k =-1
    for n_k in range(2,10):
        kmeans = KMeans(n_clusters=n_k, random_state=0).fit(filtered_df[co_ordinates.columns].values)
        kmeans.fit(filtered_df[co_ordinates.columns].values)
        labels = kmeans.labels_
        
        score = silhouette_score(filtered_df[co_ordinates.columns].values, labels)
        if score > max_score:
            max_score = score
            max_labels = labels
            max_k = n_k
    sc_1 = silhouette_score(filtered_df[co_ordinates.columns].values, filtered_df['manual_annotation'].values)
    # if max_score > 0:
    # if max_k > 1:
        # print("label: ",i,"k: ",max_k,"score: ",max_score)
        
    

ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [13]:
combined_df

,imagerow,imagecol,manual_annotation,label,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15
AAACAAGTATCTCCCA-1,7.382307,13.162828,L3,3.0,-4.518934,0.186934,-1.579811,-0.502101,0.671348,0.641350,0.321643,-0.982915,-1.298658,-0.819136,0.939768,-0.435809,-0.621682,0.595509,0.588482
AAACAATCTACTAGCA-1,-16.522121,-5.860533,L1,5.0,-1.633397,-3.833832,1.368849,2.258932,1.818374,0.790188,-1.387884,-3.464785,0.880539,0.049468,2.961622,-1.827578,0.877444,0.893352,0.614336
AAACACCAATAACTGC-1,11.761196,-13.905966,WM,0.0,21.552655,2.453866,0.134693,0.218036,2.672315,1.342351,0.246729,-0.537452,-0.812352,-0.338191,0.802174,-1.181476,1.924210,-0.287476,3.135437
AAACAGAGCGACTCCT-1,-10.846878,10.703318,L3,3.0,-3.053900,-0.924409,-3.857269,-1.544023,-0.877575,0.376134,-0.406179,0.207248,-0.451980,0.270314,-2.740054,0.360953,-0.293267,-0.700714,-0.420126
AAACAGCTTTCAGAAG-1,3.645261,-17.098599,L5,5.0,-1.619257,1.034808,1.159368,-2.779512,0.737436,-0.835721,-0.300298,1.180892,-0.731906,-4.153623,-0.342099,-1.206988,-0.484086,-1.136119,1.612747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTTCACATCCAGG-1,11.300927,-6.409192,WM,0.0,23.973508,5.234209,-0.878509,2.523938,2.460883,2.013336,-0.802269,-1.420241,1.247154,-0.523637,-0.631024,-0.214482,1.110847,-1.917133,-1.716933
TTGTTTCATTAGTCTA-1,12.289028,-10.325488,WM,0.0,19.305907,-4.021886,0.445819,0.347937,-0.672546,0.063651,-1.265542,-1.899604,-1.675924,1.196274,0.356815,-3.379339,0.588748,1.205863,1.834935
TTGTTTCCATACAACT-1,4.692478,-11.243074,L6,0.0,6.174339,-0.367159,-0.959408,-0.629316,-2.005205,-3.275628,0.266814,1.005154,1.078512,-2.839365,0.991722,-1.237173,2.573723,1.405452,1.283472
TTGTTTGTATTACACG-1,18.889032,-6.797038,WM,0.0,10.519546,-1.084939,1.415915,-1.424585,-0.918063,-1.170070,2.940803,1.221382,-0.074671,2.889727,-0.209896,1.586728,1.919082,0.681071,-3.205603


In [28]:
features = combined_df[pc_s.columns].values
features_coordinate = combined_df[['imagerow','imagecol']].values
labels = combined_df['manual_annotation'].values
features_cor = combined_df[pc_s.columns.to_list()+['imagerow','imagecol']].values
labels_ours = combined_df['label'].values

In [39]:
silhouette_cosine = silhouette_score(features,labels,metric='cosine')
silhouette_euclidean = silhouette_score(features_coordinate,labels,metric='eucidean')
print('Silhouette Score for Cosine: ',silhouette_cosine)
print('Silhouette Score for Euclidean: ',silhouette_euclidean)

ValueError: Unknown metric eucidean. Valid metrics are ['euclidean', 'l2', 'l1', 'manhattan', 'cityblock', 'braycurtis', 'canberra', 'chebyshev', 'correlation', 'cosine', 'dice', 'hamming', 'jaccard', 'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule', 'wminkowski', 'nan_euclidean', 'haversine'], or 'precomputed', or a callable

In [21]:

silhouette_cosine_cor = silhouette_score(features_cor,labels,metric='euclidean')
print('Silhouette Score for Cosine with distance: ',silhouette_cosine_cor)

Silhouette Score for Cosine with distance:  0.06999632733049578


In [26]:

silhouette_cosine_ours = silhouette_score(features,labels_ours,metric='cosine')
silhouette_euclidean_ours = silhouette_score(features_coordinate,labels_ours,metric='euclidean')
print('Silhouette Score for Cosine ours: ',silhouette_cosine_ours)
print('Silhouette Score for Euclidean ours: ',silhouette_euclidean_ours)

Silhouette Score for Cosine ours:  0.21080937700068733
Silhouette Score for Euclidean ours:  0.08949234600235363


In [72]:
def distance(cos_sim,euclidean_dist,alpha=0.9):
    return alpha * cos_sim + (1-alpha) * euclidean_dist

euclidean_dist_c = euclidean_distances(features_coordinate,features_coordinate)
cos_sim_dist_c = cosine_similarity(features,features) + 1

custom_dist_c = distance(cos_sim_dist_c,euclidean_dist_c,alpha=0.7)
np.fill_diagonal(custom_dist_c,0)
silhouette_custom = silhouette_score(custom_dist_c,labels,metric='precomputed')
print('Silhouette Score for Custom: ',silhouette_custom)

Silhouette Score for Custom:  0.03167090164064631


In [73]:
silhouette_custom_manual = silhouette_score(custom_dist_c,labels_ours,metric='precomputed')
print('Silhouette Score for Custom Manual: ',silhouette_custom_manual)

Silhouette Score for Custom Manual:  -0.04742275631833217


In [56]:
sample2 = pd.read_csv('../../../Algorithms/Unsupervised_Segmentation/Approaches/With_Scribble/Outputs/Human_DLPFC/151673/many_models_1_1_2_itr_2/Outputs/Cluster_Labels_15final.csv',index_col=0,header=0)

labels_ours = pd.concat([combined_df.drop(columns=['label']),sample2],axis=1,join='inner').fillna(-1)['label'].values

silhouette_cosine_ours = silhouette_score(features,labels_ours,metric='cosine')
silhouette_euclidean_ours = silhouette_score(features_coordinate,labels_ours,metric='euclidean')
silhouette_custom_ours = silhouette_score(custom_dist_c,labels_ours,metric='precomputed')
print('Silhouette Score for Cosine ours: ',silhouette_cosine_ours)
print('Silhouette Score for Euclidean ours: ',silhouette_euclidean_ours)
print('Silhouette Score for Custom ours: ',silhouette_custom_ours)


Silhouette Score for Cosine ours:  0.04709180024361259
Silhouette Score for Euclidean ours:  -0.04493280891503934
Silhouette Score for Custom ours:  -0.054673875134121745


### Calinski and Harabasz score

In [34]:
Calinski_Harabaz_score = calinski_harabasz_score(features,labels)
print('Calinski Harabaz Score: ',Calinski_Harabaz_score)
calinski_harabasz_score_features_coordinates = calinski_harabasz_score(features_cor,labels)
print('Calinski Harabaz Score for features coordinates: ',calinski_harabasz_score_features_coordinates)
calinski_harabasz_score_ours = calinski_harabasz_score(features,labels_ours)
print('Calinski Harabaz Score ours: ',calinski_harabasz_score_ours)
calinski_harabasz_score_features_coordinates_ours = calinski_harabasz_score(features_cor,labels_ours)
print('Calinski Harabaz Score for features coordinates ours: ',calinski_harabasz_score_features_coordinates_ours)


Calinski Harabaz Score:  867.3427564955067
Calinski Harabaz Score for features coordinates:  533.2242890984767
Calinski Harabaz Score ours:  936.5036961554136
Calinski Harabaz Score for features coordinates ours:  596.2540095448268
